In [2]:
import os, sys, glob
import pandas as pd, numpy as np, matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
from PIL import Image
from fastai import Flatten
import torchvision.models as models
import tqdm
import torch.optim as optim
import sys
stdout = sys.stdout
from sklearn.metrics import hamming_loss, accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import label_ranking_loss, coverage_error, label_ranking_average_precision_score
from sklearn.metrics import jaccard_similarity_score
from sklearn import preprocessing
from sklearn.metrics import average_precision_score

# dataset

In [15]:
class MyDataset(Dataset):
    def __init__(self, df):
        self.df = df
        for i in ['car', 'aeroplane', 'pottedplant', 'tvmonitor', 'cat', 'bus', 'bird', 'cow', 'dog',\
                'sofa', 'sheep', 'chair', 'bottle', 'person', 'bicycle', 'boat', 'motorbike', 'horse',\
                'diningtable', 'train']:
            df[i] = df[i].map({1:1, 0:1, -1:0})
        self.img_path = df.img_path
        self.labels = df.values[:,-2048-20:-2048].astype(np.int16)
        self.preprocess = T.Compose([
            T.Resize(size=(224, 224)),
            T.ToTensor(),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    def __getitem__(self, index):
        img = Image.open(self.img_path.iloc[index]).convert('RGB')
        img = self.preprocess(img)
        return img, self.labels[index]
    def __len__(self):
        return self.df.shape[0]
    def preprocess(self):
        pass

# evaluation

In [4]:

def sigmoid(x):
    return 1./(1+np.exp(-x))
def evaluate(y, y_, pivot=0.5):
    true_labels = y
    predict_probs = sigmoid(y_)
    predict_labels = (predict_probs>pivot).astype(np.int)
    #print("-----------------example based metrics--------------------------")
    #hammingLoss = hamming_loss(true_labels, predict_labels)
    accuracy = jaccard_similarity_score(true_labels, predict_labels)
    #subset_accuracy = accuracy_score(true_labels, predict_labels)
    #precision = precision_score(true_labels, predict_labels, average="samples")
    #recall = recall_score(true_labels, predict_labels, average="samples")
    f1Score = f1_score(true_labels, predict_labels, average="samples")

    # label based metrics
    #print("------------------label based metrics---------------------------")
    #micro_precision = precision_score(true_labels, predict_labels, average="micro")
    #micro_recall = recall_score(true_labels, predict_labels, average="micro")
    micro_F1_score = f1_score(true_labels, predict_labels, average="micro")
    #macro_precision = precision_score(true_labels, predict_labels, average="macro")
    #macro_recall = recall_score(true_labels, predict_labels, average="macro")
    macro_F1_score = f1_score(true_labels, predict_labels, average="macro")

    # rank metrics
    #print("--------------------rank metrics--------------------------------")
    #ranking_loss = label_ranking_loss(true_labels, predict_probs)
    #one_error = None
    #coverage = coverage_error(true_labels, predict_probs)
    MAP = average_precision_score(y_true=true_labels, y_score=predict_probs, average='macro')
    return accuracy, f1Score, micro_F1_score, macro_F1_score, MAP

# model

In [6]:
import torch
from fastai.callback import SmoothenValue
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.fc = nn.Sequential(*[
            nn.Linear(20, 1)
        ])
    def forward(self, h):
        return self.fc(h)

class MyGAN(nn.Module):
    def __init__(self):
        super(MyGAN, self).__init__()
        self.predictor = models.resnet18(pretrained=True)
        self.predictor.fc = nn.Linear(512, 20)
        self.discriminator = Discriminator()
    def forward(self, x, y):
        y_ = self.predictor(x)
        false_logit = self.discriminator((y_>0).float())
        true_logit = self.discriminator(y)
        return y_, false_logit, true_logit
class MyLoss(nn.Module):
    def __init__(self, alpha):
        super(MyLoss, self).__init__()
        self.loss1 = nn.BCEWithLogitsLoss() # for expression
        self.loss2 = nn.BCEWithLogitsLoss() # for discriminator
        self.alpha = alpha
    def forward(self, y_, y, false_logit):
        l1 = self.loss1(input=y_, target=y)
        l2 = self.loss2(input=false_logit, target=torch.ones_like(false_logit))
        return (1-self.alpha)*l1 + self.alpha*l2
class MyDiscriminatorLoss(nn.Module):
    def __init__(self):
        super(MyDiscriminatorLoss, self).__init__()
        self.loss1 = nn.BCEWithLogitsLoss()
    def forward(self, false_logit, true_logit):
        l1 = self.loss1(input=false_logit, target=torch.zeros_like(false_logit)) + \
            self.loss1(input=true_logit, target=torch.ones_like(true_logit))
        return l1/2
    
from sklearn.metrics import classification
def train_one_epoch(dl, net, loss_fn=None, optimizer=None, train_flag=True):
    loss = SmoothenValue(beta=0.9)
    if train_flag is True:
        g_loss, d_loss = loss_fn
        g_optimizer, d_optimizer = optimizer
    #pbar = tqdm.tqdm_notebook(dl, leave=False)
    y, y_ = [], []
    for batch in dl:
        batch_x, batch_y = batch[0].float().cuda(), batch[1].float().cuda()
        batch_y_, batch_false_logit, batch_true_logit = net(batch_x, batch_y)
        
        if train_flag is True:
            batch_g_loss = g_loss(batch_y_, batch_y, batch_false_logit)
            g_optimizer.zero_grad(); batch_g_loss.backward(retain_graph=True); g_optimizer.step()
            batch_d_loss = d_loss(batch_false_logit, batch_true_logit)
            d_optimizer.zero_grad(); batch_d_loss.backward(); d_optimizer.step()
            loss.add_value(batch_g_loss.item())
            #pbar.set_description(desc="loss:%.4f" % loss.smooth)
        y.append(batch_y.cpu().numpy())
        y_.append(batch_y_.detach().cpu().numpy())
    y = np.concatenate(y, axis=0)
    y_ = np.concatenate(y_, axis=0)
    return y, y_, loss.mov_avg

# train

In [27]:
def train(epochs, trn_dl, test_dl, net, loss_fn=None, optimizer=None):
    metric = {'loss':[], 'acc':[], 'f1':[], 'micf1':[], 'macf1':[], 'ap':[]}
    for i in range(epochs):
        y1, y1_, trn_loss = train_one_epoch(trn_dl, net, loss_fn, optimizer)
        trn_acc, trn_f1, trn_mic_f1, trn_mac_f1, trn_map = evaluate(y1, y1_, pivot=0.5)
        y1, y1_, test_loss = train_one_epoch(test_dl, net, train_flag=False)
        test_acc, test_f1, test_mic_f1, test_mac_f1, test_map = evaluate(y1, y1_, pivot=0.5)
        stdout.write("%d\t" % i)
        stdout.write("train: %.4f, %.4f, %.4f, %.4f, %.4f, %.4f\t" % (trn_loss, trn_acc, trn_f1, trn_mic_f1, trn_mac_f1, trn_map))
        stdout.write("test : %.4f, %.4f, %.4f, %.4f, %.4f, %.4f\n" % (test_loss, test_acc, test_f1, test_mic_f1, test_mac_f1, test_map))
        metric['loss'].append(test_loss)
        metric['acc'].append(test_acc)
        metric['f1'].append(test_f1)
        metric['micf1'].append(test_mic_f1)
        metric['macf1'].append(test_mac_f1)
        metric['ap'].append(test_map)
    return metric


In [20]:
trainval_path = '/sdadata/zzq/dataset/MultiLabel/VOC2007/feature_ds_trainval.csv'
trainval_df = pd.read_csv(trainval_path, index_col=0)
test_path = '/sdadata/zzq/dataset/MultiLabel/VOC2007/feature_ds_test.csv'
test_df = pd.read_csv(test_path, index_col=0)

In [21]:
train_ds = MyDataset(trainval_df)
test_ds = MyDataset(test_df)

In [24]:
train_dl = DataLoader(train_ds, batch_size=32, num_workers=4, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=32, num_workers=4, shuffle=False)

In [25]:
def paramtune(alpha, epochs=10):
    my_net = MyGAN().cuda()
    d_optimizer = optim.Adam(params=my_net.discriminator.parameters(), lr=1e-3)
    g_optimizer = optim.Adam(params=my_net.predictor.parameters(), lr=1e-3)
    d_loss = MyDiscriminatorLoss().cuda()
    g_loss = MyLoss(alpha).cuda()
    res = train(epochs, train_dl, test_dl, my_net, loss_fn=[g_loss, d_loss], optimizer=[g_optimizer, d_optimizer])
    return res

In [26]:
import warnings
warnings.filterwarnings('ignore')

In [1]:
alpha_set = [0.0, 0.0001,0.001,0.01,0.1,0.2,0.3, 0.4, 0.5,0.6,0.7,0.8,0.9,1.0]
times = 1
for alpha in tqdm.tqdm_notebook(alpha_set):
    res = {'loss':[], 'acc':[], 'f1':[], 'micf1':[], 'macf1':[], 'ap':[]}
    for i in range(times):
        tmp = paramtune(alpha)
        res['loss'].append(np.array(tmp['loss']).min())
        res['acc'].append(np.array(tmp['acc']).max())
        res['f1'].append(np.array(tmp['f1']).max())
        res['micf1'].append(np.array(tmp['micf1']).max())
        res['macf1'].append(np.array(tmp['macf1']).max())
        res['ap'].append(np.array(tmp['ap']).max())
    stdout.write("alpha:%.5f  " % alpha)
    stdout.write("mean-->")
    #stdout.write("loss:%.4f, " % np.array(res['loss']).mean())
    stdout.write("acc:%.4f, " % np.array(res['acc']).mean())
    stdout.write("micro-f1:%.4f, " % np.array(res['micf1']).mean())
    stdout.write("macro-f1:%.4f, " % np.array(res['macf1']).mean())
    stdout.write("map:%.4f \t" % np.array(res['ap']).mean())
    
    stdout.write("std-->")
    #stdout.write("loss:%.4f, " % np.array(res['loss']).std())
    stdout.write("acc:%.4f, " % np.array(res['acc']).std())
    stdout.write("micro-f1:%.4f, " % np.array(res['micf1']).std())
    stdout.write("macro-f1:%.4f, " % np.array(res['macf1']).std())
    stdout.write("map:%.4f\n" % np.array(res['ap']).std())
